In [1]:
import torch
import torchvision
import torch.optim as optim
from sklearn.metrics import median_absolute_error, mean_squared_error
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
from torchsummary import summary
import random
from sklearn.metrics import mean_absolute_error

In [2]:
torch.__version__

'1.4.0'

In [3]:
torchvision.__version__

'0.5.0'

In [4]:
torch.version.cuda

'10.1'

In [5]:
torch.backends.cudnn.version()

7603

In [6]:
device = 'cuda:2' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(111)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [7]:
!nvidia-smi

Sun Feb  9 16:49:32 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.26       Driver Version: 430.26       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:1D:00.0 Off |                  N/A |
| 27%   30C    P0    51W / 250W |     10MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:1E:00.0 Off |                  N/A |
| 27%   26C    P8     5W / 250W |  10665MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

# 1. Dataset

In [8]:
train_dir1 = '/mnt/disk2/data/private_data/KRISS/Train/training_data1.mat'
train_dir2 = '/mnt/disk2/data/private_data/KRISS/Train/training_data2.mat'
train_dir3 = '/mnt/disk2/data/private_data/KRISS/Train/training_data3.mat'
train_dir4 = '/mnt/disk2/data/private_data/KRISS/Train/training_data4.mat'
train_dir5 = '/mnt/disk2/data/private_data/KRISS/Train/training_data5.mat'
train_dir6 = '/mnt/disk2/data/private_data/KRISS/Train/training_data6.mat'
train_dir7 = '/mnt/disk2/data/private_data/KRISS/Train/training_data7.mat'
train_dir8 = '/mnt/disk2/data/private_data/KRISS/Train/training_data8.mat'
train_dir9 = '/mnt/disk2/data/private_data/KRISS/Train/training_data9.mat'
train_dir10 = '/mnt/disk2/data/private_data/KRISS/Train/training_data10.mat'

train_mat1 = loadmat(train_dir1)
train_mat2 = loadmat(train_dir2)
train_mat3 = loadmat(train_dir3)
train_mat4 = loadmat(train_dir4)
train_mat5 = loadmat(train_dir5)
train_mat6 = loadmat(train_dir6)
train_mat7 = loadmat(train_dir7)
train_mat8 = loadmat(train_dir8)
train_mat9 = loadmat(train_dir9)
train_mat10 = loadmat(train_dir10)

train_x_1 = train_mat1['BF0'].transpose(2,0,1)
train_x_2 = train_mat2['BF0'].transpose(2,0,1)
train_x_3 = train_mat3['BF0'].transpose(2,0,1)
train_x_4 = train_mat4['BF0'].transpose(2,0,1)
train_x_5 = train_mat5['BF0'].transpose(2,0,1)
train_x_6 = train_mat6['BF0'].transpose(2,0,1)
train_x_7 = train_mat7['BF0'].transpose(2,0,1)
train_x_8 = train_mat8['BF0'].transpose(2,0,1)
train_x_9 = train_mat9['BF0'].transpose(2,0,1)
train_x_10 = train_mat10['BF0'].transpose(2,0,1)

print(train_x_1.shape)
print(train_x_2.shape)
print(train_x_3.shape)
print(train_x_4.shape)
print(train_x_5.shape)
print(train_x_6.shape)
print(train_x_7.shape)
print(train_x_8.shape)
print(train_x_9.shape)
print(train_x_10.shape)

(13000, 51, 51)
(13000, 51, 51)
(13000, 51, 51)
(13000, 51, 51)
(13000, 51, 51)
(13000, 51, 51)
(13000, 51, 51)
(13000, 51, 51)
(13000, 51, 51)
(13000, 51, 51)


In [9]:
train_x = np.vstack([train_x_1, train_x_2, train_x_3, train_x_4,train_x_5,train_x_6,
                    train_x_7,train_x_8,train_x_9,train_x_10])
train_x.shape

(130000, 51, 51)

In [10]:
sample = len(train_x) - int(len(train_x)*0.1)
print(sample)

117000


In [11]:
## y label, 13000개 만들기

temp = []

for i in range(1000):
    for j in range(13):
        temp.append(train_mat1['Y0'][i])
for i in range(1000):
    for j in range(13):
        temp.append(train_mat2['Y0'][i])
for i in range(1000):
    for j in range(13):
        temp.append(train_mat3['Y0'][i])
        
for i in range(1000):
    for j in range(13):
        temp.append(train_mat4['Y0'][i])
for i in range(1000):
    for j in range(13):
        temp.append(train_mat5['Y0'][i])
for i in range(1000):
    for j in range(13):
        temp.append(train_mat6['Y0'][i])
for i in range(1000):
    for j in range(13):
        temp.append(train_mat7['Y0'][i])
for i in range(1000):
    for j in range(13):
        temp.append(train_mat8['Y0'][i])
for i in range(1000):
    for j in range(13):
        temp.append(train_mat9['Y0'][i])
for i in range(1000):
    for j in range(13):
        temp.append(train_mat10['Y0'][i])

train_y = np.array(temp)
print(train_y.shape)

(130000, 3)


In [12]:
class TrainDataset(Dataset):
    def __init__(self):
        
        trainX = train_x[:sample]
        trainY = train_y[:sample]
        self.trainX = torch.from_numpy(trainX).float() 
        self.trainY = torch.from_numpy(trainY).float()
        
    def __getitem__(self, index):
        x = self.trainX[index]
        y = self.trainY[index]
        return x, y
    
    def __len__(self):
        return len(self.trainX)

In [13]:
class ValDataset(Dataset):
    def __init__(self):
        
        trainX = train_x[sample:]
        trainY = train_y[sample:]
        self.trainX = torch.from_numpy(trainX).float() 
        self.trainY = torch.from_numpy(trainY).float()
        
    def __getitem__(self, index):
        x = self.trainX[index]
        y = self.trainY[index]
        return x, y
    
    def __len__(self):
        return len(self.trainX)

## 1.2. Custom dataset

In [14]:
batch_size = 32

In [15]:
train_data = TrainDataset()
val_data = ValDataset()

trainloader = DataLoader(dataset=train_data, batch_size=batch_size, num_workers=0, shuffle=True, drop_last=True)
valloader = DataLoader(dataset=val_data, batch_size=len(val_data), num_workers=0, shuffle=False, drop_last=False)

In [16]:
for nums, data in enumerate(trainloader):
    print(nums)
    x,y = data
    print(x.size(),y.size())
    if nums == 0:
        break
        
for nums, data in enumerate(valloader):
    print(nums)
    x,y = data
    print(x.size(),y.size())
    if nums == 0:
        break

0
torch.Size([32, 51, 51]) torch.Size([32, 3])
0
torch.Size([13000, 51, 51]) torch.Size([13000, 3])


# 2. Model

In [8]:
class VGGNet(nn.Module):
    def __init__(self):
        super(VGGNet, self).__init__()
        self.cnn = nn.Sequential(

        # (51,51)
        torch.nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=True),
        torch.nn.BatchNorm2d(64),    
        torch.nn.ReLU(),
        torch.nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=True),
        torch.nn.BatchNorm2d(64),    
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2), # (25,25)

        torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=True),
        torch.nn.BatchNorm2d(128),    
        torch.nn.ReLU(),
        torch.nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=True),
        torch.nn.BatchNorm2d(128),    
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2), # (12,12)

        torch.nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=True),
        torch.nn.BatchNorm2d(256),    
        torch.nn.ReLU(),
        torch.nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True),
        torch.nn.BatchNorm2d(256),    
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2), # (6,6)
            
        torch.nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=True),
        torch.nn.BatchNorm2d(512),    
        torch.nn.ReLU(),
        torch.nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),
        torch.nn.BatchNorm2d(512),    
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2),
            
        )
        # (batch, 128,6,6)
        self.linear = nn.Sequential(
        torch.nn.Linear(512*3*3, 2048),
        torch.nn.ReLU(),
        torch.nn.Linear(2048, 1024),
        torch.nn.ReLU(),
        torch.nn.Linear(1024, 512),
        torch.nn.ReLU(),
        torch.nn.Linear(512, 256),
        torch.nn.ReLU(),
        torch.nn.Linear(256,3)
        )
    def forward(self, x):
        out = self.cnn(x)
        out = self.linear(out.view(-1, 128*6*6))
        
        return out

In [9]:
train_model = VGGNet().to(device)
print(train_model)

VGGNet(
  (cnn): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1

In [10]:
!nvidia-smi

Sun Feb  9 16:49:47 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.26       Driver Version: 430.26       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:1D:00.0 Off |                  N/A |
| 27%   29C    P8    13W / 250W |     10MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:1E:00.0 Off |                  N/A |
| 27%   25C    P8     5W / 250W |  10665MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [19]:
sum(p.numel() for p in train_model.parameters() if p.requires_grad)

16882371

In [20]:
def init_params(model):
    for p in model.parameters():
        if(p.dim() > 1):
            nn.init.xavier_normal_(p)
init_params(train_model)

# 3. Train

In [21]:
total_batch = len(trainloader)
print(total_batch)

3656


In [22]:
# define cost/loss & optimizer
criterion = torch.nn.MSELoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(train_model.parameters(), lr=0.00001, weight_decay=0.000002)
num_epochs = 100

In [23]:
train_model.train()
train_iter_losses = []
train_epoch_losses = []

valid_losses = []


for epoch in range(num_epochs):
    avg_cost = 0

    for X, Y in trainloader:
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        
        output = train_model(X.unsqueeze(1))
        loss = criterion(output, Y)
        loss.backward()
        optimizer.step()
        
        train_iter_losses.append(loss)
        avg_cost += loss / total_batch
        
    train_epoch_losses.append(avg_cost)    
    
    with torch.no_grad():
        train_model.eval()
        val_loss = 0.0
        
        for x, y in valloader:
            val_x = x.to(device)
            val_y = y.to(device)
            
            val_output = train_model(val_x.unsqueeze(1))
            v_loss = criterion(val_output, val_y)
            
            val_loss += v_loss 
        valid_losses.append(val_loss)
        
#         if early_stopping.validate(val_loss):
#             break

    if epoch % 1 == 0:    
        print('Epoch:', '%04d' % (epoch + 1), 'Train Loss =''{:.4f}'.format(avg_cost), 'Val Loss=''{:.4f}'.format(val_loss))


print('Learning finished')


RuntimeError: cuDNN error: CUDNN_STATUS_NOT_SUPPORTED. This error may appear if you passed in a non-contiguous input.

In [25]:
!nvidia-smi

Sun Feb  9 16:49:14 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.26       Driver Version: 430.26       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:1D:00.0 Off |                  N/A |
| 27%   28C    P8    13W / 250W |     10MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:1E:00.0 Off |                  N/A |
| 27%   26C    P8     5W / 250W |  10665MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G